#single object annotations


In [ ]:
import os

# Define paths
annotations_dir = r"F:\VisDrone2019-DET-test-challenge\VisDrone2019-MOT-train\annotations"  # Update with actual path
output_dir = r"F:\VisDrone2019-DET-test-challenge\VisDrone2019-MOT-train\annotations-object(0)"  # Update with actual path

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process annotation files
for filename in os.listdir(annotations_dir):
    if filename.endswith(".txt"):  # Ensure only text files are processed
        input_path = os.path.join(annotations_dir, filename)
        output_path = os.path.join(output_dir, filename)
        
        with open(input_path, "r") as infile, open(output_path, "w") as outfile:
            for line in infile:
                parts = line.strip().split(",")  # Assuming CSV format with commas
                if parts[1] == "0":  # Check if object_id is 0
                    outfile.write(line)
        
        print(f"Processed: {filename}")

print("Filtering complete.")


In [ ]:
import os
import random

def remove_consecutive_frames_from_folder(annotation_folder, output_folder, min_remove=20, max_remove=50):
    """
    Dynamically removes a percentage of consecutive frames from Object 0's annotations in multiple text files.
    
    Args:
    - annotation_folder (str): Path to the folder containing annotation files.
    - output_folder (str): Path to save the modified annotation files.
    - min_remove (int): Minimum percentage of frames to remove.
    - max_remove (int): Maximum percentage of frames to remove.
    """
    os.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists

    for filename in os.listdir(annotation_folder):
        if filename.endswith(".txt"):  # Process only text files
            input_path = os.path.join(annotation_folder, filename)
            output_path = os.path.join(output_folder, filename)
            
            annotations = []
            with open(input_path, "r") as f:
                for line in f.readlines():
                    values = line.strip().split(",")  # Assuming comma-separated values
                    frame_id, object_id = int(values[0]), int(values[1])
                    if object_id == 0:
                        annotations.append(values)
            
            total_frames = len(annotations)
            if total_frames == 0:
                print(f"No object 0 annotations found in {filename}.")
                continue
            
            # Dynamically determine how many frames to remove
            remove_percentage = random.randint(min_remove, max_remove)
            frames_to_remove = int((remove_percentage / 100) * total_frames)
            
            # Select a random starting point for removal
            start_idx = random.randint(0, total_frames - frames_to_remove)
            removed_frames = [annotations[i][0] for i in range(start_idx, start_idx + frames_to_remove)]
            
            # Remove consecutive frames
            modified_annotations = annotations[:start_idx] + annotations[start_idx + frames_to_remove:]
            
            # Save modified annotations
            with open(output_path, "w") as f:
                for row in modified_annotations:
                    f.write(",".join(row) + "\n")
            
            print(f"Processed {filename}. Removed {frames_to_remove} frames starting from index {start_idx}.")
            print(f"Frames removed: {removed_frames}\n")

# Example Usage
annotation_folder = r"F:\VisDrone2019-DET-test-challenge\VisDrone2019-MOT-train\annotations-object(0)"  # Change to your actual folder path
output_folder = r"F:\VisDrone2019-DET-test-challenge\VisDrone2019-MOT-train\annotations-object(0)-randomly removed"
remove_consecutive_frames_from_folder(annotation_folder, output_folder)


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

def draw_bounding_boxes(annotation_file, image_folder, object_id=0):
    """
    Draws bounding boxes from annotation files onto corresponding image frames and displays them using Matplotlib.
    
    Args:
    - annotation_file (str): Path to the annotation text file.
    - image_folder (str): Path to the folder containing video frames as images.
    - object_id (int): Object ID to filter (default: 0).
    """
    # Read annotation file
    annotations = {}
    with open(annotation_file, "r") as f:
        for line in f.readlines():
            values = line.strip().split(",")
            frame_id = int(values[0])  # Extract frame number
            obj_id = int(values[1])  # Extract object ID
            if obj_id == object_id:
                if frame_id not in annotations:
                    annotations[frame_id] = []
                annotations[frame_id].append(list(map(int, values[2:6])))  # Store bounding box

    # Get sorted frame list from image folder
    frame_files = sorted(os.listdir(image_folder))

    for frame_file in frame_files:
        if frame_file.endswith(".jpg"):
            frame_id = int(frame_file.split(".")[0])  # Extract frame number
            image_path = os.path.join(image_folder, frame_file)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for Matplotlib

            # Draw bounding boxes if annotations exist
            plt.figure(figsize=(8, 6))
            plt.imshow(image)
            if frame_id in annotations:
                for (x, y, width, height) in annotations[frame_id]:
                    plt.gca().add_patch(plt.Rectangle((x, y), width, height, edgecolor="g", linewidth=2, fill=False))
                    plt.text(x, y - 10, f"ID: {object_id}", color="g", fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
            
            # Print frame number
            print(f"Displaying Frame Number: {frame_id}")

            # Show the frame
            plt.axis("off")
            plt.show()


# Example Usage
annotation_file = r"F:\VisDrone2019-DET-test-challenge\VisDrone2019-MOT-train\annotations-object(0)-randomly removed\uav0000295_02300_v.txt"
image_folder = r"F:\VisDrone2019-DET-test-challenge\VisDrone2019-MOT-train\sequences\uav0000295_02300_v"

draw_bounding_boxes(annotation_file, image_folder)


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

def draw_bounding_boxes(annotation_file, image_folder):
    """
    Draws bounding boxes from annotation files onto corresponding image frames and displays them using Matplotlib.
    
    Args:
    - annotation_file (str): Path to the annotation text file.
    - image_folder (str): Path to the folder containing video frames as images.
    """
    # Read annotation file
    annotations = {}
    with open(annotation_file, "r") as f:
        for line in f.readlines():
            values = line.strip().split(",")
            frame_id = int(values[0])  # Extract frame number
            obj_id = int(values[1])  # Extract object ID
            x, y, width, height = map(int, values[2:6])  # Bounding box coordinates
            
            if frame_id not in annotations:
                annotations[frame_id] = []
            annotations[frame_id].append((obj_id, x, y, width, height))  # Store bounding box and object ID

    # Get sorted frame list from image folder
    frame_files = sorted(os.listdir(image_folder))

    for frame_file in frame_files:
        if frame_file.endswith(".jpg"):
            frame_id = int(frame_file.split(".")[0])  # Extract frame number
            image_path = os.path.join(image_folder, frame_file)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for Matplotlib

            # Draw bounding boxes if annotations exist
            plt.figure(figsize=(8, 6))
            plt.imshow(image)
            if frame_id in annotations:
                for (obj_id, x, y, width, height) in annotations[frame_id]:
                    plt.gca().add_patch(plt.Rectangle((x, y), width, height, edgecolor="g", linewidth=2, fill=False))
                    plt.text(x, y - 10, f"ID: {obj_id}", color="g", fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
            
            # Print frame number
            print(f"Displaying Frame Number: {frame_id}")

            # Show the frame
            plt.axis("off")
            plt.show()

# Example Usage
annotation_file = r"F:\VisDrone2019-DET-test-challenge\VisDrone2019-MOT-train\annotations\uav0000013_01073_v.txt"
image_folder = r"F:\VisDrone2019-DET-test-challenge\VisDrone2019-MOT-train\sequences\uav0000013_01073_v"

draw_bounding_boxes(annotation_file, image_folder)
